# Imports

In [1]:
import nibabel as nib
import numpy as np

from clustering_layer import ClusteringLayer # T-SNE algorithm which is incorporated at the end of the network

from keras import backend as K
from keras.layers import Activation, Input, Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv3D, Cropping3D
from keras.layers.core import Permute, Reshape
from keras.layers.merge import concatenate
from keras.regularizers import l1_l2
from keras.models import Model, load_model
K.set_image_dim_ordering('th')
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler
from keras.utils import np_utils

import itertools
import matplotlib.pyplot as plt
from sklearn.feature_extraction.image import extract_patches as sk_extract_patches

from functions import (config, generate_model, get_filename, get_set_name, read_data, read_vol, save_vol, 
                       extract_patches, build_set, generate_indexes, reconstruct_volume, target_distribution, 
                       DSC, save_vol_modif, generate_indexes_modif, reconstruct_volume_modif, read_vol_modif, 
                       extract_patches_modif, build_set_modif, restartkernel) 
                       # Functions designed to load images, generate a dataset, and save images

(num_classes, patience, model_filename, csv_filename, nb_epoch, validation_split, class_mapper, 
 class_mapper_inv, PATCH_SHAPE, EXTRACTION_SHAPE, n_known) = config() 
# Config variables set in functions, can be overridden in the Notebook

n_known = 11 # Number of images used for the training set

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


Using TensorFlow backend.


# 1. Initial segmentation

In [8]:
T1_vols = np.empty((n_known-1, 144, 192, 256))
T2_vols = np.empty((n_known-1, 144, 192, 256))
label_vols = np.empty((n_known-1, 144, 192, 256))
for case_idx in range(1, n_known) :
    T1_vols[(case_idx - 1), :, :, :] = read_vol(case_idx, 'T1')
    T2_vols[(case_idx - 1), :, :, :] = read_vol(case_idx, 'T2')
    label_vols[(case_idx - 1), :, :, :] = read_vol(case_idx, 'label')

## 1.2 Pre-processing

In [9]:
## Intensity normalisation (zero mean and unit variance)
T1_mean = T1_vols.mean()
T1_std = T1_vols.std()
T1_vols = (T1_vols - T1_mean) / T1_std
T2_mean = T2_vols.mean()
T2_std = T2_vols.std()
T2_vols = (T2_vols - T2_mean) / T2_std

# Combine labels of BG and CSF
for class_idx in class_mapper :
    label_vols[label_vols == class_idx] = class_mapper[class_idx]
label_vols = label_vols.astype('float16')

## 1.3 Data preparation

In [5]:
x_train, y_train = build_set(T1_vols, T2_vols, label_vols, (3, 9, 3))

Finished segmentation of case # 0
Finished segmentation of case # 1
Finished segmentation of case # 2
Finished segmentation of case # 3
Finished segmentation of case # 4
Finished segmentation of case # 5
Finished segmentation of case # 6
Finished segmentation of case # 7
Finished segmentation of case # 8
Finished segmentation of case # 9


## 1.4 Configure callbacks

In [6]:
# The learning rate linearly decreases over the epochs
def lr_schedule(ep):
    lr = 1e-3
    lr *= (nb_epoch - ep)/float(nb_epoch)
    print('lr: ', lr)
    return lr

# Model checkpoint to save the training results
checkpointer = ModelCheckpoint(
    filepath=model_filename.format(1),
    verbose=1,
    save_best_only=True,
    save_weights_only=True)

# CSVLogger to save the training results in a csv file
csv_logger = CSVLogger(csv_filename.format(1), separator=';')

# Learning rate scheduler
learning_rate_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpointer, csv_logger, learning_rate_scheduler]

## 1.5 Training

In [7]:
model = generate_model(num_classes)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 2, 27, 27, 27 0                                            
__________________________________________________________________________________________________
conv3d_27 (Conv3D)              (None, 25, 25, 25, 2 1375        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_27 (BatchNo (None, 25, 25, 25, 2 100         conv3d_27[0][0]                  
__________________________________________________________________________________________________
p_re_lu_27 (PReLU)              (None, 25, 25, 25, 2 390625      batch_normalization_27[0][0]     
__________________________________________________________________________________________________
conv3d_28 

In [8]:
# Build model
model = generate_model(num_classes)

# Train model
model.fit(
    x_train,
    y_train,
    epochs=nb_epoch,
    validation_split=validation_split,
    verbose=1,
    callbacks=callbacks)

# freeing space
del x_train
del y_train

Train on 96749 samples, validate on 24188 samples
('lr: ', 0.001)
Epoch 1/20
96749/96749 [==============================] - 1318s 14ms/step - loss: 0.4764 - categorical_accuracy: 0.8505 - val_loss: 0.3390 - val_categorical_accuracy: 0.8897
('lr: ', 0.00095)
Epoch 2/20
96749/96749 [==============================] - 1316s 14ms/step - loss: 0.2936 - categorical_accuracy: 0.9065 - val_loss: 0.2751 - val_categorical_accuracy: 0.9065
('lr: ', 0.0009000000000000001)
Epoch 3/20
96749/96749 [==============================] - 1316s 14ms/step - loss: 0.2530 - categorical_accuracy: 0.9181 - val_loss: 0.2849 - val_categorical_accuracy: 0.9000
('lr: ', 0.00085)
Epoch 4/20
96749/96749 [==============================] - 1315s 14ms/step - loss: 0.2349 - categorical_accuracy: 0.9232 - val_loss: 0.2425 - val_categorical_accuracy: 0.9160
('lr: ', 0.0008)
Epoch 5/20
96749/96749 [==============================] - 1315s 14ms/step - loss: 0.2209 - categorical_accuracy: 0.9275 - val_loss: 0.2486 - val_categori

## 1.6 Classification

In [10]:
# Load best model according to the validation set
model = generate_model(num_classes)
model.load_weights(model_filename.format(1))

In [11]:
# Save segmentations of the test set
for case_idx in range(6,24) :
    T1_test_vol = read_vol(case_idx, 'T1')[:144, :192, :256]
    T2_test_vol = read_vol(case_idx, 'T2')[:144, :192, :256]

    x_test = np.zeros((6916, 2, PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE))
    x_test[:, 0, :, :, :] = extract_patches(T1_test_vol, 
                                            patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                            extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))
    x_test[:, 1, :, :, :] = extract_patches(T2_test_vol, 
                                            patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                            extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))

    x_test[:, 0, :, :, :] = (x_test[:, 0, :, :, :] - T1_mean) / T1_std
    x_test[:, 1, :, :, :] = (x_test[:, 1, :, :, :] - T2_mean) / T2_std

    pred = model.predict(x_test, verbose=2)
    pred = np.argmax(pred, axis=2)
    pred = pred.reshape((len(pred), EXTRACTION_SHAPE, EXTRACTION_SHAPE, 
                                         EXTRACTION_SHAPE))
    segmentation = reconstruct_volume(pred, (144, 192, 256))

    csf = np.logical_and(segmentation == 0, T1_test_vol != 0)
    segmentation[segmentation == 2] = 250
    segmentation[segmentation == 1] = 150
    segmentation[csf] = 10

    save_vol(segmentation, case_idx)

    print("Finished segmentation of case # {}".format(case_idx))

Finished segmentation of case # 6
Finished segmentation of case # 7
Finished segmentation of case # 8
Finished segmentation of case # 9
Finished segmentation of case # 10
Finished segmentation of case # 11
Finished segmentation of case # 12
Finished segmentation of case # 13
Finished segmentation of case # 14
Finished segmentation of case # 15
Finished segmentation of case # 16
Finished segmentation of case # 17
Finished segmentation of case # 18
Finished segmentation of case # 19
Finished segmentation of case # 20
Finished segmentation of case # 21
Finished segmentation of case # 22
Finished segmentation of case # 23
Done with Step 1


In [12]:
# Print segmentation scores of 5 images
for case_idx in range(6,11):
    im1 = read_data(case_idx, 'label', 'datasets')
    im2 = read_data(case_idx, 'label', 'results')
    im1 = im1.get_data()
    im2 = im2.get_data()

    x = []
    print('case_idx: ', case_idx)
    for i in [10, 150, 250]:
        x.append(DSC(im1, im2, i))
        print(i, ':', DSC(im1, im2, i))
    print('average : ', np.mean(x))

('case_idx: ', 6)
(10, ':', 0.9682849707585975)
(150, ':', 0.9398487499247111)
(250, ':', 0.9282338514851699)
('average : ', 0.9454558573894928)
('case_idx: ', 7)
(10, ':', 0.966125824136991)
(150, ':', 0.9417523289756555)
(250, ':', 0.931827292663947)
('average : ', 0.9465684819255312)
('case_idx: ', 8)
(10, ':', 0.9696613913192658)
(150, ':', 0.9432344694894719)
(250, ':', 0.9292195374811059)
('average : ', 0.9473717994299479)
('case_idx: ', 9)
(10, ':', 0.9418968079110519)
(150, ':', 0.9067204616477362)
(250, ':', 0.9025529804386385)
('average : ', 0.9170567499991421)
('case_idx: ', 10)
(10, ':', 0.9531112812084919)
(150, ':', 0.9073739758366893)
(250, ':', 0.8606722000204939)
('average : ', 0.9070524856885585)


In [6]:
# Save probabilities results for the whole dataset (train and test)
# Differently from previously, for each voxel, 3 probabilities are recorded
# This data is necessary as we use soft labels for semi-supervised training
for case_idx in range(1, 24) :
    T1_test_vol = read_vol(case_idx, 'T1')[:144, :192, :256]
    T2_test_vol = read_vol(case_idx, 'T2')[:144, :192, :256]

    x_test = np.zeros((6916, 2, PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE))
    x_test[:, 0, :, :, :] = extract_patches(T1_test_vol, 
                                            patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                            extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))
    x_test[:, 1, :, :, :] = extract_patches(T2_test_vol, 
                                            patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                            extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))

    x_test[:, 0, :, :, :] = (x_test[:, 0, :, :, :] - T1_mean) / T1_std
    x_test[:, 1, :, :, :] = (x_test[:, 1, :, :, :] - T2_mean) / T2_std

    pred = model.predict(x_test, verbose=2)
    pred = target_distribution(pred)
    pred = pred.reshape((len(pred), EXTRACTION_SHAPE, EXTRACTION_SHAPE, 
                                         EXTRACTION_SHAPE, num_classes))
    segmentation = reconstruct_volume_modif(pred, (144, 192, 256, num_classes))
    
    save_vol_modif(segmentation, case_idx)

    print("Finished segmentation of case # {}".format(case_idx))

print("Done with Step 1")

Finished segmentation of case # 1
Finished segmentation of case # 2
Finished segmentation of case # 3
Finished segmentation of case # 4
Finished segmentation of case # 5
Finished segmentation of case # 6
Finished segmentation of case # 7
Finished segmentation of case # 8
Finished segmentation of case # 9
Finished segmentation of case # 10
Finished segmentation of case # 11
Finished segmentation of case # 12
Finished segmentation of case # 13
Finished segmentation of case # 14
Finished segmentation of case # 15
Finished segmentation of case # 16
Finished segmentation of case # 17
Finished segmentation of case # 18
Finished segmentation of case # 19
Finished segmentation of case # 20
Finished segmentation of case # 21
Finished segmentation of case # 22
Finished segmentation of case # 23
Done with Step 1


In [12]:
# Freeing space before set 2
%reset -f

# 2. Semi-supervised step

In [1]:
# The step 2 can be run until the score doesn't increase further
numiter = 10
for ite in range(numiter):
    
    import nibabel as nib
    import numpy as np

    from clustering_layer import ClusteringLayer # T-SNE algorithm which is incorporated at the end of the network

    from keras import backend as K
    from keras.layers import Activation, Input, Dropout, BatchNormalization
    from keras.layers.advanced_activations import PReLU
    from keras.layers.convolutional import Conv3D, Cropping3D
    from keras.layers.core import Permute, Reshape
    from keras.layers.merge import concatenate
    from keras.regularizers import l1_l2
    from keras.models import Model, load_model
    K.set_image_dim_ordering('th')
    from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler
    from keras.utils import np_utils

    import itertools
    import matplotlib.pyplot as plt
    from sklearn.feature_extraction.image import extract_patches as sk_extract_patches

    from functions import (config, generate_model, get_filename, get_set_name, read_data, read_vol, save_vol, 
                           extract_patches, build_set, generate_indexes, reconstruct_volume, target_distribution, 
                           DSC, save_vol_modif, generate_indexes_modif, reconstruct_volume_modif, read_vol_modif, 
                           extract_patches_modif, build_set_modif, restartkernel) 
                           # Functions designed to load images, generate a dataset, and save images

    (num_classes, patience, model_filename, csv_filename, nb_epoch, validation_split, class_mapper, 
     class_mapper_inv, PATCH_SHAPE, EXTRACTION_SHAPE, n_known) = config() 
    # Config variables set in functions, can be overridden in the Notebook

    n_known = 11 # Number of images used for the training set
    
    from keras.utils import to_categorical
    
    # sets of training and testing images
    sure = range(n_known-1)
    unsure = range(n_known-1, 23)
    
    T1_vols = np.empty((23, 144, 192, 256), dtype='float32')
    T2_vols = np.empty((23, 144, 192, 256), dtype='float32')
    label_vols = np.empty((23, 144, 192, 256, 3), dtype='float16')

    # For the testing set, we load the segmentations results instead of the (unknown) true labels
    for case_idx in range(n_known, 24) :
        loc = 'results'

        T1_vols[(case_idx - 1), :, :, :] = read_vol_modif(case_idx, 'T1')[:144, :192, :256, 0]
        T2_vols[(case_idx - 1), :, :, :] = read_vol_modif(case_idx, 'T2')[:144, :192, :256, 0]
        label_vols[(case_idx - 1), :, :, :] = read_vol_modif(case_idx, 'label', loc)[:144, :192, :256]
        print(case_idx)

    # For the training set, we load the true labels
    for case_idx in range(1, n_known) :
        loc = 'datasets'

        T1_vols[(case_idx - 1), :, :, :] = read_vol_modif(case_idx, 'T1')[:144, :192, :256, 0]
        T2_vols[(case_idx - 1), :, :, :] = read_vol_modif(case_idx, 'T2')[:144, :192, :256, 0]
        x = read_vol_modif(case_idx, 'label', loc)[:144, :192, :256]
        for class_idx in class_mapper :
            x[x == class_idx] = class_mapper[class_idx]
        label_vols[(case_idx - 1)] = to_categorical(x)
        print(case_idx)
        
    T1_vols = T1_vols.astype('float32')
    T2_vols = T2_vols.astype('float32')
    label_vols = label_vols.astype('float16')
    
    ## Intensity normalisation (zero mean and unit variance)
    T1_mean = T1_vols.mean()
    T1_std = T1_vols.std()
    T1_vols = (T1_vols - T1_mean) / T1_std
    T2_mean = T2_vols.mean()
    T2_std = T2_vols.std()
    T2_vols = (T2_vols - T2_mean) / T2_std
    
    # Build sets
    x_sure, y_sure = build_set_modif(T1_vols[sure], T2_vols[sure], label_vols[sure], n_known, False, (3, 9, 3))
    x_unsure, y_unsure = build_set_modif(T1_vols[unsure], T2_vols[unsure], label_vols[unsure], n_known, True)

    x_train = np.vstack((x_sure, x_unsure))
    y_train = np.vstack((y_sure, y_unsure))

    # Freeing space
    del x_sure
    del x_unsure
    del y_sure
    del y_unsure
    
    # The learning rate linearly decreases over the epochs
    def lr_schedule(ep):
        lr = 1e-3
        lr *= (nb_epoch - ep)/float(nb_epoch)
        print 'lr: ', lr
        return lr

    # Model checkpoint to save the training results
    checkpointer = ModelCheckpoint(
        filepath=model_filename.format(2),
        verbose=1,
        save_best_only=True,
        save_weights_only=True)

    # CSVLogger to save the training results in a csv file
    csv_logger = CSVLogger(csv_filename.format(2), separator=';')

    # Learning rate scheduler
    learning_rate_scheduler = LearningRateScheduler(lr_schedule)

    callbacks = [checkpointer, csv_logger, learning_rate_scheduler]
    
    # Build model
    model = generate_model(num_classes)
    
    # Train model
    model.fit(
        x_train,
        y_train,
        epochs=nb_epoch,
        validation_split=validation_split,
        verbose=1,
        callbacks=callbacks)

    # freeing space
    del x_train
    del y_train

    # Load best model
    model = generate_model(num_classes)
    model.load_weights(model_filename.format(2))
    
    # Save segmentation results for 5 images
    for case_idx in range(6, 11) :
        T1_test_vol = read_vol(case_idx, 'T1')[:144, :192, :256]
        T2_test_vol = read_vol(case_idx, 'T2')[:144, :192, :256]

        x_test = np.zeros((6916, 2, PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE))
        x_test[:, 0, :, :, :] = extract_patches(T1_test_vol, 
                                                patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                                extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))
        x_test[:, 1, :, :, :] = extract_patches(T2_test_vol, 
                                                patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                                extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))

        x_test[:, 0, :, :, :] = (x_test[:, 0, :, :, :] - T1_mean) / T1_std
        x_test[:, 1, :, :, :] = (x_test[:, 1, :, :, :] - T2_mean) / T2_std

        pred = model.predict(x_test, verbose=2)
        pred_classes = np.argmax(pred, axis=2)
        pred_classes = pred_classes.reshape((len(pred_classes), EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))
        segmentation = reconstruct_volume(pred_classes, (144, 192, 256))

        csf = np.logical_and(segmentation == 0, T1_test_vol != 0)
        segmentation[segmentation == 2] = 250
        segmentation[segmentation == 1] = 150
        segmentation[csf] = 10

        save_vol(segmentation, case_idx, 'refined-results')

        print "Finished segmentation of case # {}".format(case_idx)
        
    # Print Dice score for these images
    for case_idx in range(6, 11):
        im1 = read_data(case_idx, 'label', 'datasets')
        im2 = read_data(case_idx, 'label', 'refined-results')
        im1 = im1.get_data()
        im2 = im2.get_data()

        x = []
        print('case_idx: ', case_idx)
        for i in [10, 150, 250]:
            x.append(DSC(im1, im2, i))
            print(i, ':', DSC(im1, im2, i))
        print('average : ', np.mean(x))
    
    # Save probabilities results to be used as soft labels for the next iteration
    for case_idx in range(n_known,24) :
        T1_test_vol = read_vol(case_idx, 'T1')[:144, :192, :256]
        T2_test_vol = read_vol(case_idx, 'T2')[:144, :192, :256]

        x_test = np.zeros((6916, 2, PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE))
        x_test[:, 0, :, :, :] = extract_patches(T1_test_vol, 
                                                patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                                extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))
        x_test[:, 1, :, :, :] = extract_patches(T2_test_vol, 
                                                patch_shape=(PATCH_SHAPE, PATCH_SHAPE, PATCH_SHAPE), 
                                                extraction_step=(EXTRACTION_SHAPE, EXTRACTION_SHAPE, EXTRACTION_SHAPE))

        x_test[:, 0, :, :, :] = (x_test[:, 0, :, :, :] - T1_mean) / T1_std
        x_test[:, 1, :, :, :] = (x_test[:, 1, :, :, :] - T2_mean) / T2_std

        pred = model.predict(x_test, verbose=2)
        pred = target_distribution(pred)
        pred = pred.reshape((len(pred), EXTRACTION_SHAPE, EXTRACTION_SHAPE, 
                                             EXTRACTION_SHAPE, num_classes))
        segmentation = reconstruct_volume_modif(pred, (144, 192, 256, num_classes))

        save_vol_modif(segmentation, case_idx)

        print("Finished segmentation of case # {}".format(case_idx))
        
    # Freeing space
    %reset -f

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


Using TensorFlow backend.


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
1
2
3
4
5
2747
Finished segmentation of case # 0
2334
Finished segmentation of case # 1
2443
Finished segmentation of case # 2
2122
Finished segmentation of case # 3
2603
Finished segmentation of case # 4
1292
Finished segmentation of case # 0
1339
Finished segmentation of case # 1
1331
Finished segmentation of case # 2
1459
Finished segmentation of case # 3
1277
Finished segmentation of case # 4
1291
Finished segmentation of case # 5
1205
Finished segmentation of case # 6
1499
Finished segmentation of case # 7
1321
Finished segmentation of case # 8
1335
Finished segmentation of case # 9
1507
Finished segmentation of case # 10
1108
Finished segmentation of case # 11
1335
Finished segmentation of case # 12
1306
Finished segmentation of case # 13
1518
Finished segmentation of case # 14
1293
Finished segmentation of case # 15
1529
Finished segmentation of case # 16
1309
Finished segmentation of case # 17
Train on 29202 samples, validate on